In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
!pip install --upgrade ipywidgets



In [3]:
# =========================================================
# Frequency-Aware DeepFake Detection (FFT + CNN)
# Dataset: Extracted Faces (Real / Fake folders)
# Limit images per video/folder
# =========================================================

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import timm
from tqdm import tqdm

# =======================
# 1. PATH CONFIG
# =======================

BASE_DIR = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion"

DATASET_DIR = os.path.join(BASE_DIR, "extracted_frames_Cross_vit")
REAL_DIR = os.path.join(DATASET_DIR, "real")
FAKE_DIR = os.path.join(DATASET_DIR, "fake")

RESULT_DIR = os.path.join(BASE_DIR, "results_frequency")
os.makedirs(RESULT_DIR, exist_ok=True)

LABELS = {"real": 0, "fake": 1}

# Max faces per video/subfolder
MAX_FACES_PER_VIDEO = 50

# =======================
# 2. FFT TRANSFORMATION
# =======================

def fft_transform(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    fft = np.fft.fft2(gray)
    fft_shift = np.fft.fftshift(fft)
    magnitude = np.log(np.abs(fft_shift) + 1)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = magnitude.astype(np.uint8)
    magnitude = cv2.cvtColor(magnitude, cv2.COLOR_GRAY2RGB)
    return magnitude

# =======================
# 3. DATASET CLASS
# =======================

class FrequencyDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None, max_faces_per_video=50):
        self.samples = []
        self.transform = transform
        self.max_faces_per_video = max_faces_per_video

        def collect_images(base_dir, label):
            for root, dirs, _ in os.walk(base_dir):
                for subfolder in dirs:
                    subfolder_path = os.path.join(root, subfolder)
                    files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
                    files = files[:self.max_faces_per_video]  # Limit number of faces per video
                    for f in files:
                        self.samples.append((os.path.join(subfolder_path, f), label))

        collect_images(real_dir, LABELS["real"])
        collect_images(fake_dir, LABELS["fake"])

        print(f"✅ Total images loaded: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        freq_img = fft_transform(img)

        if self.transform:
            freq_img = self.transform(image=freq_img)["image"]

        return freq_img, torch.tensor(label)

# =======================
# 4. TRANSFORMS
# =======================

transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])

# =======================
# 5. DATASET & SPLIT
# =======================

dataset = FrequencyDataset(
    real_dir=REAL_DIR,
    fake_dir=FAKE_DIR,
    transform=transform,
    max_faces_per_video=MAX_FACES_PER_VIDEO
)

labels_list = [dataset[i][1].item() for i in range(len(dataset))]

train_idx, val_idx = train_test_split(
    range(len(dataset)),
    test_size=0.2,
    random_state=42,
    stratify=labels_list
)

train_set = torch.utils.data.Subset(dataset, train_idx)
val_set   = torch.utils.data.Subset(dataset, val_idx)

train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=8, shuffle=False)

# =======================
# 6. MODEL
# =======================

device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model(
    "resnet18",
    pretrained=True,
    num_classes=2
).to(device)

# =======================
# 7. TRAINING SETUP
# =======================

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 10

# =======================
# 8. TRAINING LOOP
# =======================

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{EPOCHS}]")
    for x, y in pbar:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    print(f"✅ Epoch {epoch+1} finished | Avg Loss: {total_loss/len(train_loader):.4f}")

# =======================
# 9. EVALUATION
# =======================

model.eval()
preds, labels, probs = [], [], []

with torch.no_grad():
    for x, y in tqdm(val_loader, desc="Validation"):
        x = x.to(device)
        out = model(x)

        prob = torch.softmax(out, dim=1)[:, 1]
        pred = out.argmax(1).cpu().numpy()

        preds.extend(pred)
        labels.extend(y.numpy())
        probs.extend(prob.cpu().numpy())

acc = accuracy_score(labels, preds)
auc = roc_auc_score(labels, probs)

print("✅ Validation Accuracy:", acc)
print("✅ Validation AUC:", auc)

# =======================
# 10. SAVE RESULTS
# =======================

with open(os.path.join(RESULT_DIR, "results.txt"), "w") as f:
    f.write(f"Accuracy: {acc}\n")
    f.write(f"AUC: {auc}\n")

# =======================
# 11. INFERENCE FUNCTION
# =======================

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    freq = fft_transform(img)
    freq = transform(image=freq)["image"].unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        out = model(freq)
        prob = torch.softmax(out, dim=1)[0, 1].item()

    return "FAKE" if prob > 0.5 else "REAL", prob

# Example usage:
# label, conf = predict_image("frame_0000.jpg")
# print(label, conf)


✅ Total images loaded: 47565


Epoch [1/10]: 100%|██████████| 4757/4757 [07:35<00:00, 10.43it/s, loss=0.1295]


✅ Epoch 1 finished | Avg Loss: 0.3793


Epoch [2/10]: 100%|██████████| 4757/4757 [07:35<00:00, 10.44it/s, loss=0.1482]


✅ Epoch 2 finished | Avg Loss: 0.2945


...

✅ Epoch 9 finished | Avg Loss: 0.0531


Epoch [10/10]: 100%|██████████| 4757/4757 [08:31<00:00,  9.31it/s, loss=0.0013]


✅ Epoch 10 finished | Avg Loss: 0.0429


Validation: 100%|██████████| 1190/1190 [01:12<00:00, 16.47it/s]


✅ Validation Accuracy: 0.901292967518133
✅ Validation AUC: 0.9230383972221654


In [7]:
# =========================================================
# Frequency-Aware DeepFake Detection (FFT + CNN)
# Dataset: Extracted Faces (Real / Fake folders)
# Limit images per video/folder
# =========================================================

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import timm
from tqdm import tqdm

# =======================
# 1. PATH CONFIG
# =======================

BASE_DIR = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion"

DATASET_DIR = os.path.join(BASE_DIR, "extracted_frames_Cross_vit")
REAL_DIR = os.path.join(DATASET_DIR, "real")
FAKE_DIR = os.path.join(DATASET_DIR, "fake")

RESULT_DIR = os.path.join(BASE_DIR, "results_frequency")
os.makedirs(RESULT_DIR, exist_ok=True)

LABELS = {"real": 0, "fake": 1}

# Max faces per video/subfolder
MAX_FACES_PER_VIDEO = 80

# =======================
# 2. FFT TRANSFORMATION
# =======================

def fft_transform(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    fft = np.fft.fft2(gray)
    fft_shift = np.fft.fftshift(fft)
    magnitude = np.log(np.abs(fft_shift) + 1)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = magnitude.astype(np.uint8)
    magnitude = cv2.cvtColor(magnitude, cv2.COLOR_GRAY2RGB)
    return magnitude

# =======================
# 3. DATASET CLASS
# =======================

class FrequencyDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None, max_faces_per_video=80):
        self.samples = []
        self.transform = transform
        self.max_faces_per_video = max_faces_per_video

        def collect_images(base_dir, label):
            for root, dirs, _ in os.walk(base_dir):
                for subfolder in dirs:
                    subfolder_path = os.path.join(root, subfolder)
                    files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
                    files = files[:self.max_faces_per_video]  # Limit number of faces per video
                    for f in files:
                        self.samples.append((os.path.join(subfolder_path, f), label))

        collect_images(real_dir, LABELS["real"])
        collect_images(fake_dir, LABELS["fake"])

        print(f"✅ Total images loaded: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        freq_img = fft_transform(img)

        if self.transform:
            freq_img = self.transform(image=freq_img)["image"]

        return freq_img, torch.tensor(label)

# =======================
# 4. TRANSFORMS
# =======================

transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])

# =======================
# 5. DATASET & SPLIT
# =======================

dataset = FrequencyDataset(
    real_dir=REAL_DIR,
    fake_dir=FAKE_DIR,
    transform=transform,
    max_faces_per_video=MAX_FACES_PER_VIDEO
)

labels_list = [dataset[i][1].item() for i in range(len(dataset))]

train_idx, val_idx = train_test_split(
    range(len(dataset)),
    test_size=0.2,
    random_state=42,
    stratify=labels_list
)

train_set = torch.utils.data.Subset(dataset, train_idx)
val_set   = torch.utils.data.Subset(dataset, val_idx)

train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=8, shuffle=False)

# =======================
# 6. MODEL
# =======================

device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model(
    "resnet18",
    pretrained=True,
    num_classes=2
).to(device)

# =======================
# 7. TRAINING SETUP
# =======================

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 10

# =======================
# 8. TRAINING LOOP
# =======================

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{EPOCHS}]")
    for x, y in pbar:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    print(f"✅ Epoch {epoch+1} finished | Avg Loss: {total_loss/len(train_loader):.4f}")

# =======================
# 9. EVALUATION
# =======================

model.eval()
preds, labels, probs = [], [], []

with torch.no_grad():
    for x, y in tqdm(val_loader, desc="Validation"):
        x = x.to(device)
        out = model(x)

        prob = torch.softmax(out, dim=1)[:, 1]
        pred = out.argmax(1).cpu().numpy()

        preds.extend(pred)
        labels.extend(y.numpy())
        probs.extend(prob.cpu().numpy())

acc = accuracy_score(labels, preds)
auc = roc_auc_score(labels, probs)

print("✅ Validation Accuracy:", acc)
print("✅ Validation AUC:", auc)

# =======================
# 10. SAVE RESULTS
# =======================

with open(os.path.join(RESULT_DIR, "results.txt"), "w") as f:
    f.write(f"Accuracy: {acc}\n")
    f.write(f"AUC: {auc}\n")

# =======================
# 11. INFERENCE FUNCTION
# =======================

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    freq = fft_transform(img)
    freq = transform(image=freq)["image"].unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        out = model(freq)
        prob = torch.softmax(out, dim=1)[0, 1].item()

    return "FAKE" if prob > 0.5 else "REAL", prob




✅ Total images loaded: 75568


Epoch [1/10]: 100%|██████████| 7557/7557 [12:24<00:00, 10.15it/s, loss=0.3117]


✅ Epoch 1 finished | Avg Loss: 0.3668


Epoch [2/10]: 100%|██████████| 7557/7557 [11:57<00:00, 10.54it/s, loss=0.1446]


✅ Epoch 2 finished | Avg Loss: 0.2806


...

✅ Epoch 9 finished | Avg Loss: 0.0516


Epoch [10/10]: 100%|██████████| 7557/7557 [15:58<00:00,  7.88it/s, loss=0.0221]


✅ Epoch 10 finished | Avg Loss: 0.0437


Validation: 100%|██████████| 1890/1890 [02:42<00:00, 11.62it/s]

✅ Validation Accuracy: 0.9241762604208019
✅ Validation AUC: 0.9482802277049276
